# Define a single test

In [1]:
EXAMPLES_FOLDER = "../examples"

In [2]:
import os, pprint
[f for f in os.listdir(EXAMPLES_FOLDER) if not f.startswith('.')]

['change_case',
 'multivariate',
 'sacurine',
 'workflow-test-suite-full.yml',
 'workflow-test-suite-min.yml',
 'workflows.json']

In [3]:
suite_conf_filename = os.path.join(EXAMPLES_FOLDER, "workflow-test-suite-min.yml")

In [4]:
import ruamel.yaml as yaml
with open(suite_conf_filename, "r") as fp:
    data = yaml.load(fp)
    print yaml.round_trip_dump(data, indent=2, block_seq_indent=3)

workflows:
  change_case:
    expected:
      OutputText: change_case/expected_output
    inputs:
      InputText: change_case/input
    file: change_case/workflow.ga
  multivariate:
    expected:
      variableMetadata_out: multivariate/variableMetadata_out
      sampleMetadata_out: multivariate/sampleMetadata_out
    inputs:
      DataMatrix: multivariate/dataMatrix.tsv
      SampleMetadata: multivariate/sampleMetadata.tsv
      VariableMetadata: multivariate/variableMetadata.tsv
    params:
      3:
        predI: '1'
        respC: gender
        orthoI: NA
        testL: 'FALSE'
    file: multivariate/workflow.ga



<center>![wf-changecase-workflow](../images/ChangeCaseWorkflow.png "Workflow ChangeCase")</center>
<center>**Figure 1.** Workflow *"ChangeCase"*</center>

As Fig. 1 shows, the workflow has one input and one output. To test it, both the input and the output must be uniquely identified. Typically, Galaxy identifies them using **Names** and **Labels**, respectively.  For our sample workflow, the identifiers are:

* Name **"InputText"** for the input (Fig. 2);
* Label **"OutputText"** for the output (Fig. 3).

<div>
<div style="display: inline-block; width: 50%; margin: 0 auto;">
<center>![wf-changecase-inputdataset](../images/ChangeCaseInputDataset.png "Workflow “Input Dataset“")</center>
<center>**Figure 2.** Workflow input: *"Input Dataset"*</center>
</div>
<div style="display: inline-block">
<center>![wf-changecase-outputdataset](../images/ChangeCaseOutputDataset.png "Workflow “Output Dataset“")</center>
<center>**Figure 3.** Workflow output: *"output1"*</center>
</div>
</div>

In [5]:
from wft4galaxy import Workflow

In [6]:
help(Workflow)

Help on class Workflow in module wft4galaxy:

class Workflow
 |  Display workflow information which are relevant to configure a workflow test.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, definition, inputs, params, outputs)
 |  
 |  show_inputs(self)
 |      Print workflow inputs to the standard output.
 |  
 |  show_outputs(self)
 |      Print workflow outputs (indexed by workflow step) to the standard output.
 |  
 |  show_params(self)
 |      Print parameters needed by workflow tools to the standard output.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  load(filename, tools_folder='.tools', galaxy_url=None, galaxy_api_key=None)
 |      Return the :class:`Workflow` instance related to the workflow defined in ``filename``
 |      
 |      :type filename: str
 |      :param filename: the path of the ``.ga`` workflow definition
 |      
 |      :type tools_folder: str
 |      :param tools_folder: optio

In [7]:
wf = Workflow.load(os.path.join(EXAMPLES_FOLDER, "change_case/workflow.ga"))

In [8]:
wf.show_inputs()

-  InputText 


In [9]:
 wf.show_params()

'1':
  cols: ''  # Change case of columns
  delimiter: TAB # Delimited by
  casing: up # To



In [10]:
 wf.show_outputs()

'1': OutputText


In [11]:
from wft4galaxy import WorkflowTestConfiguration

In [12]:
help(WorkflowTestConfiguration)

Help on class WorkflowTestConfiguration in module wft4galaxy:

class WorkflowTestConfiguration
 |  A representation of the configuration of a workflow test.
 |  
 |  :type base_path: str
 |  :param base_path: base path for workflow and dataset files (the current working path is assumed as default)
 |  
 |  :type workflow_filename: str
 |  :param workflow_filename: the path (relative to ``base_path``) of the file containing
 |      the workflow definition (i.e., the ``.ga`` file which can be downloaded from a Galaxy server)
 |  
 |  :type name: str
 |  :param name: a name for the workflow test
 |  
 |  :type inputs: dict
 |  :param inputs: a dictionary which defines the mapping between a workflow input and a test dataset.
 |  
 |         :Example:
 |  
 |              {"DataMatrix": "dataMatrix.tsv"} where `DataMatrix` is the name of a workflow input
 |              and `dataMatrix.tsv` is the file containing the dataset to be used as input for the workflow test.
 |  
 |  :type params: 

In [13]:
# main params to create a WorkflowTestConfiguration: base_path, inputs, params, expected_outputs

In [14]:
base_path = os.path.join(EXAMPLES_FOLDER, "change_case")

In [15]:
workflow_filename = "workflow.ga"

In [16]:
inputs = {"InputText": {"file": "input"}}

In [17]:
expected_outputs = {"OutputText": {"file": "expected_output"}}

In [18]:
test_conf = WorkflowTestConfiguration(base_path=base_path, workflow_filename=workflow_filename, inputs=inputs, expected_outputs=expected_outputs)

In [19]:
print test_conf

WorkflowTestConfig: name=3a0860ab-a820-11e6-aa71-a45e60c4fc6b, file=workflow.ga, inputs=[InputText], expected_outputs=[OutputText]


In [20]:
from wft4galaxy import WorkflowTestRunner

In [21]:
test_runner = WorkflowTestRunner.new_instance(test_conf)

In [22]:
test_result = test_runner.run_test(enable_logger=True)

2016-11-11 16:05:18,910 INFO: Create a history '_WorkflowTestHistory_423a30dc-a820-11e6-b5e5-a45e60c4fc6b' (id: u'52d6bdfafedbb5e5')
2016-11-11 16:05:20,387 INFO: Workflow '_WorkflowTest_Change Case (imported from API)' (id: d448712f90897b61) running ...
2016-11-11 16:05:29,599 INFO: Workflow '_WorkflowTest_Change Case (imported from API)' (id: d448712f90897b61) executed
2016-11-11 16:05:29,602 INFO: Checking test output: ...
2016-11-11 16:05:29,692 INFO: Checking test output: DONE


In [23]:
print("Test %s:\n\t - workflow: [%s] \n\t - results: %r" % \
      (test_result.test_id, test_result.workflow.name, test_result.results))

Test 423a30dc-a820-11e6-b5e5-a45e60c4fc6b:
	 - workflow: [_WorkflowTest_Change Case (imported from API)] 
	 - results: {u'OutputText': True}


In [24]:
print "Outputs: ", test_result.results.keys()

Outputs:  [u'OutputText']


In [25]:
test_result.passed(), test_result.failed()

(True, False)

In [26]:
test_result.check_output("OutputText")

True

In [27]:
test_result.workflow

Workflow({u'inputs': {u'0': {u'uuid': u'00bd81ab-17f6-4722-98b6-3217a8144b9c', u'value': u'', u'label': u'InputText'}}, u'name': u'_WorkflowTest_Change Case (imported from API)', u'tags': [], u'deleted': False, u'latest_workflow_uuid': u'29d5dd98-c529-433e-91ba-94bfa714da3f', u'annotation': None, u'url': u'/api/workflows/d448712f90897b61', u'steps': {u'1': Step({u'tool_id': u'ChangeCase', u'tool_version': u'1.0.0', u'annotation': None, u'input_steps': {u'input': {u'step_output': u'output', u'source_step': u'0'}}, u'tool_inputs': {u'input': {u'__class__': u'RuntimeValue'}, u'delimiter': u'TAB', u'casing': u'up', u'cols': u'c1,c2'}, u'type': u'tool', u'id': u'1'}), u'0': Step({u'tool_id': None, u'tool_version': None, u'annotation': None, u'input_steps': {}, u'tool_inputs': {u'name': u'InputText'}, u'type': u'data_input', u'id': u'0'})}, u'published': False, u'owner': u'phenomenal', u'model_class': u'StoredWorkflow', u'id': u'd448712f90897b61'})